<a href="https://colab.research.google.com/github/caleb-cecil/AI-Book-Recommendation-System/blob/main/AI_Book_Recommendation_SystemV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mountings drive and import statements

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from sklearn.metrics.pairwise import cosine_similarity

#dont change this
path = "/content/drive/MyDrive/Colab Notebooks/470BookData"



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
print(path)

/content/drive/MyDrive/Colab Notebooks/470BookData


# Load datasets, set low_memory to False to avoid memory issues
books_df = pd.read_csv(os.path.join(path, "Books.csv"), low_memory=False)
ratings_df = pd.read_csv(os.path.join(path, "Ratings.csv"), low_memory=False)
users_df = pd.read_csv(os.path.join(path, "Users.csv"), low_memory=False)

In [ ]:
books_df = pd.read_csv(path + "/Books.csv", low_memory=False)
ratings_df = pd.read_csv(path + "/Ratings.csv", low_memory=False)
users_df = pd.read_csv(path + "/Users.csv", low_memory=False)


In [ ]:
ratings_df.head()


,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [ ]:
users_df.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [ ]:
print(books_df.shape)

(271360, 8)


In [ ]:
print(ratings_df.shape)

(1149780, 3)


In [ ]:
print(users_df.shape)

(278858, 3)


In [ ]:
books_ratings_df = pd.merge(books_df, ratings_df, on='ISBN')
books_ratings_df = books_ratings_df.drop(columns=['Image-URL-S', 'Image-URL-M', 'Image-URL-L'])
books_ratings_df = books_ratings_df[books_ratings_df['Book-Rating'] > 0]

books_ratings_df.head()


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,User-ID,Book-Rating
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,8,5
3,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,11676,8
5,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,67544,8
8,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,116866,9
9,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,123629,9


In [ ]:
# keep only users with 50+ ratings and books with 50+ ratings to reduce risk of crashing
user_counts = books_ratings_df['User-ID'].value_counts()
book_counts = books_ratings_df['Book-Title'].value_counts()

print(user_counts , book_counts)

filtered_books_ratings_df = books_ratings_df[
    (books_ratings_df['User-ID'].isin(user_counts[user_counts >= 5].index)) &
    (books_ratings_df['Book-Title'].isin(book_counts[book_counts >= 5].index))
    ]

User-ID
11676     6943
98391     5691
189835    1899
153662    1845
23902     1180
          ... 
116107       1
220131       1
149322       1
216883       1
259679       1
Name: count, Length: 68091, dtype: int64 Book-Title
The Lovely Bones: A Novel                 707
Wild Animus                               581
The Da Vinci Code                         494
The Secret Life of Bees                   406
The Nanny Diaries: A Novel                393
                                         ... 
Jedi Quest - Path to Truth (Star Wars)      1
Walter's Purple Heart                       1
Trapped Fear Street 51                      1
Heather Takes the Reins                     1
From One to One Hundred                     1
Name: count, Length: 135567, dtype: int64


# Adding genre to data set

In [ ]:
# Adding genre to the books in the dataset.

def assign_genre(title):
  genre_mapping = {

          "The Kitchen God's Wife": "Fiction",
          "The Testament": "Legal Thriller",
          "Beloved (Plume Contemporary Fiction)": "Historical Fiction",
          "Wild Animus": "Philosophical Fiction",
          "Airframe": "Techno-Thriller",
          "Timeline": "Science Fiction",
          "To Kill a Mockingbird": "Classic",
          "Seabiscuit: An American Legend": "Biography",
          "Pigs in Heaven": "Contemporary Fiction",
          "From the Corner of His Eye": "Suspense",
          "Isle of Dogs": "Crime Fiction",
          "The Street Lawyer": "Legal Thriller",
          "Breathing Lessons": "Contemporary Fiction",
          "The Joy Luck Club": "Fiction",
          "The Tao of Pooh": "Philosophy",
          "Seabiscuit": "Biography",
          "The Catcher in the Rye": "Classic",
          "Midnight in the Garden of Good and Evil: A Savannah Story": "Non-Fiction/True Crime",
          "Pretend You Don't See Her": "Mystery",
          "Fast Women": "Romance",
          "The Pillars of the Earth": "Historical Fiction",
          "The Five People You Meet in Heaven": "Philosophy/Fiction",
          "The Beach House": "Mystery",
          "Angels & Demons": "Thriller",
          "The Girl Who Loved Tom Gordon": "Horror",
          "The Sum of All Fears": "Political Thriller",
          "One Hundred Years of Solitude": "Magical Realism",
          "Little Altars Everywhere: A Novel": "Contemporary Fiction",
          "Before I Say Good-Bye": "Suspense",
          "Slow Waltz in Cedar Bend": "Romance",
          "Atonement : A Novel": "Historical Fiction",
          "Rebecca": "Gothic Fiction",
          "Contact": "Science Fiction",
          "Wish You Well": "Historical Fiction",
          "The Hitchhiker's Guide to the Galaxy": "Science Fiction/Comedy",
          "Bridget Jones's Diary": "Chick Lit",
          "The God of Small Things": "Literary Fiction",
          "The Big Bad Wolf: A Novel": "Crime Fiction",
          "Four Blind Mice": "Crime Fiction",
          "Congo": "Adventure",
          "Memoirs of a Geisha": "Historical Fiction",
          "Plainsong (Vintage Contemporaries)": "Contemporary Fiction",
          "All That Remains (Kay Scarpetta Mysteries)": "Mystery",
          "The Brethren": "Legal Thriller",
          "The King of Torts": "Legal Thriller",
          "Bleachers": "Sports Fiction",
          "Sisterhood of the Traveling Pants": "Young Adult",
          "Roses Are Red (Alex Cross Novels)": "Crime Fiction",
          "The Rescue": "Romance",
          "Manhattan Hunt Club": "Suspense",
          "Pet Sematary": "Horror",
          "Girl with a Pearl Earring": "Historical Fiction",
          "Deception Point": "Thriller",
          "Tis : A Memoir": "Biography",
          "Chicken Soup for the Soul (Chicken Soup for the Soul)": "Inspirational",
          "Wicked: The Life and Times of the Wicked Witch of the West": "Fantasy",
          "Life of Pi": "Adventure/Philosophy",
          "Lies and the Lying Liars Who Tell Them: A Fair and Balanced Look at the Right": "Satire",
          "Secret History": "Psychological Fiction",
          "Chicken Soup for the Woman's Soul": "Inspirational",
          "Wicked: The Life and Times of the Wicked Witch of the West": "Fantasy",
    "Life of Pi": "Adventure, Fantasy, Drama",
    "Lies and the Lying Liars Who Tell Them: A Fair and Balanced Look at the Right": "Politics, Humor",
    "Secret History": "Mystery, Psychological Thriller",
    "Chicken Soup for the Woman's Soul": "Self-help, Inspirational",
    "The Amber Spyglass (His Dark Materials, Book 3)": "Fantasy",
    "The Golden Compass (His Dark Materials, Book 1)": "Fantasy",
    "The Subtle Knife (His Dark Materials, Book 2)": "Fantasy",
    "Animal Farm": "Political Allegory, Dystopian",
    "Catch 22": "Satire, War, Fiction",
    "A Man in Full": "Literary Fiction",
    "A Painted House": "Historical Fiction, Coming of Age",
    "The Perfect Storm : A True Story of Men Against the Sea": "Non-fiction, Adventure",
    "Vinegar Hill (Oprah's Book Club)": "Historical Fiction, Family Drama",
    "The Hunt for Red October": "Thriller, Spy Fiction",
    "The Secret Life of Bees": "Historical Fiction, Coming of Age",
    "Digital Fortress : A Thriller": "Thriller, Technology",
    "The Firm": "Legal Thriller, Mystery",
    "The Client": "Legal Thriller, Mystery",
    "Heart of the Sea (Irish Trilogy)": "Historical Fiction",
    "The Little Prince": "Children's Literature, Philosophical Fiction",
    "Jonathan Livingston Seagull": "Philosophical Fiction, Motivational",
    "Moonlight Becomes You": "Romance, Mystery",
    "Divine Secrets of the Ya-Ya Sisterhood": "Women's Fiction, Family Drama",
    "The Lovely Bones: A Novel": "Literary Fiction, Supernatural",
    "Hannibal": "Psychological Thriller, Crime Fiction",
    "The Four Agreements: A Practical Guide to Personal Freedom": "Self-help, Personal Development",
    "A Widow for One Year": "Literary Fiction",
    "Empire Falls": "Literary Fiction",
    "Black and Blue": "Crime Fiction, Domestic Drama",
    "She's Come Undone": "Literary Fiction, Family Drama",
    "Nickel and Dimed: On (Not) Getting By in America": "Non-fiction, Sociology",
    "Left Behind: A Novel of the Earth's Last Days": "Christian Fiction, Dystopian",
    "The Shining": "Horror",
    "The Diary of Ellen Rimbauer: My Life at Rose Red": "Horror, Fiction",
    "The Bean Trees": "Literary Fiction",
    "Daughter of Fortune": "Historical Fiction",
    "The Red Tent (Bestselling Backlist)": "Historical Fiction",
    "The Nanny Diaries: A Novel": "Women's Fiction, Satire",
    "Postmortem": "Thriller, Crime Fiction",
    "Three Junes": "Literary Fiction, Family Drama",
    "Back Roads": "Literary Fiction, Domestic Drama",
    "In Her Shoes": "Women's Fiction, Family Drama",
    "The Kiss": "Romance",
    "Strangers": "Thriller, Mystery",
    "A Thousand Acres (Ballantine Reader's Circle)": "Literary Fiction",
    "Four Blondes": "Satire, Women's Fiction",
    "The Chamber": "Thriller, Legal Fiction",
    "The Partner": "Thriller, Legal Fiction",
    "Chosen Prey": "Thriller, Crime Fiction",
    "Night Sins": "Thriller, Mystery",
    "The Vampire Lestat (Vampire Chronicles, Book II)": "Fantasy, Horror",
    "The Mummy or Ramses the Damned": "Fantasy, Horror",
    "Daisy Fay and the Miracle Man": "Women's Fiction, Coming of Age",
    "Angela's Ashes: A Memoir": "Memoir, Non-fiction",
    "Fast Food Nation: The Dark Side of the All-American Meal": "Non-fiction, Sociology",
    "One for the Money (Stephanie Plum Novels)": "Mystery, Crime Fiction",
    "Along Came a Spider (Alex Cross Novels)": "Thriller, Mystery",
    "Who Moved My Cheese?": "Self-help, Motivational",
    "Where the Heart Is": "Women's Fiction, Family Drama",
    "The Girls' Guide to Hunting and Fishing": "Women's Fiction, Humor",
    "Icy Sparks": "Literary Fiction",
    "The Da Vinci Code": "Thriller, Mystery",
    "A Map of the World": "Literary Fiction",
    "Bridget Jones : The Edge of Reason": "Romance, Humor",
    "The Pilot's Wife : A Novel": "Women's Fiction, Literary Fiction",
    "Big Trouble": "Satire, Humor",
    "Snow Falling on Cedars": "Historical Fiction, Mystery",
    "Skipping Christmas": "Humor, Christmas Fiction",
    "The Reader": "Literary Fiction, Romance",
    "Daddy's Little Girl": "Thriller, Mystery",
    "Message in a Bottle": "Romance, Drama",
    "The Guardian": "Romance, Thriller",
    "Nights in Rodanthe": "Romance, Drama",
    "Like Water for Chocolate": "Romance, Magical Realism",
    "The Two Towers (The Lord of the Rings, Part 2)": "Fantasy",
    "The Villa": "Romance, Mystery",
    "Suzanne's Diary for Nicholas": "Romance, Drama",
    "2nd Chance": "Thriller, Mystery",
    "The Bad Beginning (A Series of Unfortunate Events, Book 1)": "Children's Literature, Fantasy",
    "Me Talk Pretty One Day": "Humor, Memoir",
    "The Door to December": "Thriller, Mystery",
    "About a Boy": "Literary Fiction, Comedy",
    "Dreamcatcher": "Horror, Sci-Fi",
    "Smilla's Sense of Snow": "Mystery, Thriller",
    "The Giver": "Dystopian, Children's Literature",
    "The Dead Zone": "Horror, Thriller",
    "Ender's Game": "Science Fiction",
    "A Time to Kill": "Legal Thriller, Drama",
    "Chasing the Dime": "Thriller, Crime Fiction",
    "Charming Billy": "Literary Fiction",
    "Wuthering Heights": "Classical Literature, Romance",
    "Middlesex: A Novel": "Literary Fiction, Family Saga",
    "The Rainmaker": "Legal Thriller, Drama",
    "The Amazing Adventures of Kavalier & Clay": "Literary Fiction, Historical Fiction",
    "The Phantom Tollbooth": "Children's Literature, Fantasy",
    "The Princess Bride": "Fantasy, Adventure, Romance",
    "Fight Club": "Psychological Fiction, Satire",
    "SHIPPING NEWS": "Literary Fiction, Drama",
    "The Smoke Jumper": "Historical Fiction, Adventure",
    "Anne of Green Gables": "Children's Literature, Classic",
    "Fall On Your Knees": "Family Drama, Literary Fiction",
    "While I Was Gone": "Literary Fiction",
    "The House of the Spirits": "Magical Realism, Historical Fiction",
    "Interpreter of Maladies": "Short Stories, Literary Fiction",
    "Dragonfly in Amber": "Historical Fiction, Romance",
    "Naked": "Memoir, Humor",
    "Kiss the Girls": "Thriller, Crime Fiction",
    "Different Seasons": "Short Stories, Horror",
    "The Witching Hour (Lives of the Mayfair Witches)": "Fantasy, Horror",
    "Jack & Jill (Alex Cross Novels)": "Thriller, Mystery",
    "The House on Mango Street": "Literary Fiction, Coming of Age",
    "The Perks of Being a Wallflower": "Young Adult, Coming of Age",
    "American Gods": "Fantasy, Mythology",
    "Red Dragon": "Thriller, Crime Fiction",
    "Jurassic Park": "Science Fiction, Thriller",
    "Neverwhere": "Fantasy, Urban Fantasy",
    "The Thorn Birds": "Historical Fiction, Romance",
    "Angelas Ashes": "Memoir, Non-fiction",
    "Memoirs of a Geisha": "Historical Fiction",
    "Choke": "Dark Humor, Literary Fiction",
    "Lamb : The Gospel According to Biff, Christ's Childhood Pal": "Historical Fiction, Satire",
    "The Color of Water": "Memoir, Non-fiction",
    "Anne Frank: The Diary of a Young Girl": "Memoir, Non-fiction",
    "The Stand: Complete and Uncut": "Horror, Dystopian",
    "Shopaholic Takes Manhattan": "Chick Lit, Romance",
    "Confessions of a Shopaholic": "Chick Lit, Romance",
    "The Cider House Rules": "Literary Fiction",
    "Good Omens": "Fantasy, Humor",
    "Fatal Voyage": "Thriller, Mystery",
    "Shock": "Thriller, Crime Fiction",
    "The Poisonwood Bible": "Historical Fiction",
    "A Lesson Before Dying": "Literary Fiction, Historical Fiction",
    "Jewel": "Romance, Fiction",
    "Their Eyes Were Watching God": "Literary Fiction, African American Literature",
    "A Prayer for Owen Meany": "Literary Fiction, Fiction",
    "Cold Sassy Tree": "Historical Fiction",
    "The Shipping News": "Literary Fiction, Drama",
    "The No. 1 Ladies' Detective Agency": "Mystery, Fiction",
    "Shopgirl": "Romance, Literary Fiction",
    "Breath, Eyes, Memory": "Literary Fiction, Drama",
    "A Walk in the Woods": "Non-fiction, Adventure",
    "The Silmarillion": "Fantasy",
    "Songs in Ordinary Time": "Literary Fiction",
    "The Last Juror": "Mystery, Thriller",
    "Summer Sisters": "Women's Fiction",
    "At Home in Mitford": "Christian Fiction",
    "Bel Canto": "Literary Fiction, Romance",
    "All Around the Town": "Thriller, Mystery",
    "Faking It": "Romance, Women's Fiction",
    "The Fountainhead": "Philosophical Fiction",
    "The Loop": "Thriller, Fiction",
    "Good in Bed": "Women's Fiction",
    "We'll Meet Again": "Romance, Historical Fiction",
    "How to Make an American Quilt": "Romance, Women's Fiction",
    "Ellen Foster": "Literary Fiction",
    "The Hours": "Literary Fiction",
    "Cold Mountain": "Historical Fiction, War",
    "Fried Green Tomatoes at the Whistle Stop Cafe": "Literary Fiction, Women's Fiction",
    "The Last Time They Met": "Literary Fiction, Romance",
    "The Green Mile": "Fantasy, Drama",
    "A Heartbreaking Work of Staggering Genius": "Memoir, Literary Fiction",
    "The Dress Lodger": "Historical Fiction",
    "The Fellowship of the Ring": "Fantasy",
    "Watership Down": "Fantasy, Animal Fiction",
    "The Blue Nowhere": "Thriller, Cyberpunk",
    "A Year in Provence": "Non-fiction, Memoir",
    "When the Wind Blows": "Fantasy, Thriller",
    "The Sparrow": "Science Fiction, Literary Fiction",
    "I Capture the Castle": "Literary Fiction, Coming of Age",
    "What to Expect When You're Expecting": "Non-fiction, Parenting",
    "What to Expect the First Year": "Non-fiction, Parenting",
    "The Alchemist": "Fantasy, Philosophy",
    "Miss Julia Speaks Her Mind": "Women's Fiction, Humor",
    "Big Stone Gap": "Women's Fiction",
    "Back When We Were Grownups": "Women's Fiction",
    "Black Notice": "Thriller, Mystery",
    "One for the Money": "Mystery, Crime Fiction",
    "Lord of the Flies": "Classic, Fiction",
    "The Alienist": "Historical Fiction, Mystery",
    "Notes from a Small Island": "Travel",
    "Into Thin Air": "Non-fiction, Memoir",
      }

  # Return the genre from the mapping or "Unknown" if the title is not found
  return genre_mapping.get(title, "Unknown")




def get_genre(title):
  return assign_genre(title)
filtered_books_ratings_df['genre'] = filtered_books_ratings_df['Book-Title'].apply(assign_genre)
filtered_books_ratings_df

<ipython-input-41-f60c6fbf2878>:258: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_books_ratings_df['genre'] = filtered_books_ratings_df['Book-Title'].apply(assign_genre)


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,User-ID,Book-Rating,genre
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,8,5,Unknown
3,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,11676,8,Unknown
5,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,67544,8,Unknown
8,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,116866,9,Unknown
12,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,210926,9,Unknown
...,...,...,...,...,...,...,...,...
1030544,0399231129,If You Come Softly,Jacqueline Woodson,1998,Speak,244804,9,Unknown
1030545,0523416873,The Presence,Yvonne MacManus,1982,Pinnacle Books,244804,10,Unknown
1030574,0811822907,Hush Little Baby,Sylvia Long,2002,Chronicle Books,245088,8,Unknown
1030982,0395647398,"The Two Towers (The Lord of the Rings, Part 2)",J. R. R. Tolkien,1992,Houghton Mifflin,259901,10,Fantasy


# Optional Popularity recommendation section

In [ ]:
# returns the average rating for each book can be used to return basic popularity based recommendations
avg_rating_df = filtered_books_ratings_df.groupby('Book-Title')['Book-Rating'].mean().reset_index()
avg_rating_df.rename(columns={'Book-Rating': 'avg_rating'}, inplace=True)

avg_rating_df

,Book-Title,avg_rating
0,Earth Prayers From around the World: 365 Pray...,6.666667
1,Murder of a Sleeping Beauty (Scumble River My...,7.000000
2,!Yo!,8.000000
3,'Salem's Lot,7.857143
4,01-01-00: A Novel of the Millennium,6.000000
...,...,...
13719,iI Paradiso Degli Orchi,8.500000
13720,murder@maggody.com : An Arly Hanks Mystery (Ar...,7.600000
13721,one hundred years of solitude,7.400000
13722,stardust,8.090909


# Collaborative filtering with genre using cosine similarity
returns books that are similar based on genre and how similar users rated those books

In [ ]:
x = filtered_books_ratings_df.groupby('User-ID').count()['Book-Rating'] > 100
experienced_users = x[x].index
experienced_users.size

144

In [ ]:
filtered_rating = filtered_books_ratings_df[filtered_books_ratings_df['User-ID'].isin(experienced_users)]

filtered_rating.size

222368

In [ ]:
# Code used from Alanskoshy
# We will incorporate genre and use cosine similarity instead of euclidian distance
# hopefully this will improve preformance
y = filtered_rating.groupby('Book-Title').count()['Book-Rating'] >= 5
famous_books = y[y].index

famous_books.size

1459

In [ ]:
final_ratings = filtered_rating[filtered_rating['Book-Title'].isin(famous_books)]

pt = final_ratings.pivot_table(index='Book-Title', columns='User-ID', values='Book-Rating')

pt.fillna(0, inplace=True)

pt


User-ID,4017,6251,6543,6575,7346,11676,12982,13552,16634,16795,...,258185,258534,261829,264321,265115,265889,270713,271448,273113,274061
Book-Title,,,,,,,,,,,,,,,,,,,,,
10 Lb. Penalty,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16 Lighthouse Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1984,0.0,0.0,0.0,0.0,8.0,10.0,0.0,0.0,0.0,8.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,9.0,0.0,0.0,9.0,0.0,0.0,0.0,9.0,...,9.0,6.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
204 Rosewood Lane,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,10.0,8.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yukon Ho!,0.0,10.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zoya,0.0,0.0,0.0,0.0,0.0,7.0,0.0,10.0,0.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
book_genre = filtered_books_ratings_df[['Book-Title', 'genre']].drop_duplicates('Book-Title').set_index('Book-Title')

unique_genres = set()

for g in book_genre['genre']:

  genre = [genre.strip() for genre in g.split(',')]
  unique_genres.update(genre)

unique_genres = list(unique_genres)
print(unique_genres)


['Classical Literature', 'Inspirational', 'Romance', 'Political Thriller', 'Political Allegory', 'Family Drama', 'Sociology', 'Spy Fiction', 'Philosophical Fiction', "Children's Literature", 'Self-help', 'Legal Thriller', 'Satire', 'Non-fiction', 'Adventure', 'Science Fiction', 'Techno-Thriller', 'Animal Fiction', 'African American Literature', 'Comedy', 'Domestic Drama', 'Motivational', 'Coming of Age', 'Young Adult', 'Chick Lit', 'Supernatural', 'Suspense', 'Fiction', 'Contemporary Fiction', 'Philosophy/Fiction', 'Biography', 'Fantasy', 'Technology', 'Politics', "Women's Fiction", 'Sci-Fi', 'Non-Fiction/True Crime', 'Magical Realism', 'Psychological Fiction', 'Mystery', 'Memoir', 'Dystopian', 'Humor', 'Legal Fiction', 'Crime Fiction', 'Classic', 'Personal Development', 'Unknown', 'War', 'Drama', 'Short Stories', 'Mythology', 'Travel', 'Urban Fantasy', 'Gothic Fiction', 'Horror', 'Psychological Thriller', 'Literary Fiction', 'Historical Fiction', 'Science Fiction/Comedy', 'Sports Fict

In [ ]:
genre_matrix = pd.DataFrame(0, index=book_genre.index, columns=unique_genres)

# Fill in the matrix
for book_title, row in book_genre.iterrows():
    # Split the genre string for this book
    genres = [genre.strip() for genre in row['genre'].split(',')]
    for gr in genres:
        if gr in genre_matrix.columns:
            genre_matrix.at[book_title, gr] = 1

In [ ]:
common_books = pt.index.intersection(genre_matrix.index)
pt = pt.loc[common_books]
genre_matrix = genre_matrix.loc[common_books]

In [ ]:
combined_features = pd.concat([pt, genre_matrix], axis=1)
combined_features

,4017,6251,6543,6575,7346,11676,12982,13552,16634,16795,...,Literary Fiction,Historical Fiction,Science Fiction/Comedy,Sports Fiction,Family Saga,Parenting,Christmas Fiction,Thriller,Dark Humor,Philosophy
Book-Title,,,,,,,,,,,,,,,,,,,,,
10 Lb. Penalty,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
16 Lighthouse Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1984,0.0,0.0,0.0,0.0,8.0,10.0,0.0,0.0,0.0,8.0,...,0,0,0,0,0,0,0,0,0,0
1st to Die: A Novel,0.0,0.0,9.0,0.0,0.0,9.0,0.0,0.0,0.0,9.0,...,0,0,0,0,0,0,0,0,0,0
204 Rosewood Lane,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yukon Ho!,0.0,10.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
Zoya,0.0,0.0,0.0,0.0,0.0,7.0,0.0,10.0,0.0,6.0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
similarity_scores = cosine_similarity(combined_features)

In [ ]:
similarity_scores.shape
similarity_scores

array([[1.        , 0.00357343, 0.00180387, ..., 0.00307484, 0.19063788,
        0.00307032],
       [0.00357343, 1.        , 0.00201115, ..., 0.00342818, 0.15811229,
        0.00342313],
       [0.00180387, 0.00201115, 1.        , ..., 0.20593445, 0.1766397 ,
        0.13996758],
       ...,
       [0.00307484, 0.00342818, 0.20593445, ..., 1.        , 0.27210256,
        0.16789414],
       [0.19063788, 0.15811229, 0.1766397 , ..., 0.27210256, 1.        ,
        0.12694279],
       [0.00307032, 0.00342313, 0.13996758, ..., 0.16789414, 0.12694279,
        1.        ]])

In [ ]:
def recommend(book_name):
    index = np.where(combined_features.index==book_name)[0][0]
    similar_items = sorted(list(enumerate(similarity_scores[index])),key=lambda x:x[1],reverse=True)[1:10]

    data = []
    for i in similar_items:
        item = []
        temp_df = books_df[books_df['Book-Title'] == combined_features.index[i[0]]]
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Title'].values))
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Author'].values))
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Image-URL-M'].values))

        data.append(item)

    return data

In [ ]:
recommend('')

[['The Secret Life of Bees',
  'Sue Monk Kidd',
  'http://images.amazon.com/images/P/0142001740.01.MZZZZZZZ.jpg'],
 ['Shattered',
  'Dean R. Koontz',
  'http://images.amazon.com/images/P/0425099334.01.MZZZZZZZ.jpg'],
 ['TickTock',
  'Dean R. Koontz',
  'http://images.amazon.com/images/P/034538430X.01.MZZZZZZZ.jpg'],
 ['The Great Gatsby',
  'F. Scott Fitzgerald',
  'http://images.amazon.com/images/P/185326041X.01.MZZZZZZZ.jpg'],
 ['The Book of Questions',
  'Pablo Neruda',
  'http://images.amazon.com/images/P/1556590415.01.MZZZZZZZ.jpg'],
 ['Prodigal Summer',
  'Barbara Kingsolver',
  'http://images.amazon.com/images/P/0060199652.01.MZZZZZZZ.jpg'],
 ['Back When We Were Grownups: A Novel',
  'ANNE TYLER',
  'http://images.amazon.com/images/P/0375412530.01.MZZZZZZZ.jpg'],
 ['No Second  Chance',
  'Harlan Coben',
  'http://images.amazon.com/images/P/0525947299.01.MZZZZZZZ.jpg'],
 ['The Conspiracy Club',
  'JONATHAN KELLERMAN',
  'http://images.amazon.com/images/P/0345452577.01.MZZZZZZZ.jpg